In [4]:
from src.resolve_init import GetResolve
from pprint import pprint

resolve = GetResolve()
project_manager = resolve.GetProjectManager()
project = project_manager.GetCurrentProject()
media_storage = resolve.GetMediaStorage()
media_pool = project.GetMediaPool()
root_folder = media_pool.GetRootFolder()
current_timeline = project.GetCurrentTimeline()


def get_subfolder_by_name(subfolder_name: str):
    all_subfolder = root_folder.GetSubFolderList()
    subfolder_dict = {
        subfolder.GetName(): subfolder for subfolder in all_subfolder
    }
    return subfolder_dict.get(subfolder_name, "")


def get_video_clips_in_date_group_source_folder(date_group: str) -> list:
    source_clip_list = []

    date_group_folder_structure = get_subfolder_by_name(
        date_group
    ).GetSubFolderList()  # type: ignore

    for folder in date_group_folder_structure:
        if folder.GetName() == "Source":
            for each_video_reel in folder.GetSubFolderList():
                reel_clips = each_video_reel.GetClipList()
                for clip in reel_clips:
                    source_clip_list.append(clip)

    return source_clip_list


def create_timeline(timeline_name: str, width: int, height: int) -> bool:
    media_pool.CreateEmptyTimeline(timeline_name)
    current_timeline = project.GetCurrentTimeline()
    current_timeline.SetSetting("useCustomSettings", "1")
    current_timeline.SetSetting("timelineResolutionWidth", str(width))
    current_timeline.SetSetting("timelineResolutionHeight", str(height))
    return current_timeline.SetSetting("timelineFrameRate", str(24))


def get_clips_by_clip_color(date_group: str, clip_color: str) -> list:
    source_clip_list = get_video_clips_in_date_group_source_folder(date_group)
    return [
        clip
        for clip in source_clip_list
        if clip.GetClipProperty("Clip Color") == clip_color
    ]


def get_scene(date_group: str, clip_color: str) -> set:
    scene_list = []
    for clip in get_clips_by_clip_color(date_group, clip_color):
        scene_list.append(clip.GetClipProperty("Scene"))
    return set(scene_list)


def get_clips_by_scene(date_group: str, clip_color: str, scene: str) -> list:
    return [
        clip
        for clip in get_clips_by_clip_color(date_group, clip_color)
        if clip.GetClipProperty("Scene") == scene
    ]


def get_clips_and_its_associated_SceneShotTake_info(date_group: str) -> dict:
    all_clips = [clip for clip in get_clips_by_clip_color(date_group, "Pink")]

    clips_scene_shot_take_dict = {}
    for clip in all_clips:
        clips_scene_shot_take_dict[clip] = {}
        clips_scene_shot_take_dict[clip]["Scene"] = clip.GetClipProperty(
            "Scene"
        )
        clips_scene_shot_take_dict[clip]["Shot"] = clip.GetClipProperty("Shot")
        clips_scene_shot_take_dict[clip]["Take"] = clip.GetClipProperty("Take")

    return clips_scene_shot_take_dict


def append_to_timeline(date_group: str):
    for scene in get_scene(date_group, "Pink"):
        current_scene_clips: list = get_clips_by_scene(
            date_group, "Pink", scene
        )
        sorted_clips = sorted(
            current_scene_clips,
            key=lambda clip: clip.GetClipProperty("Shot"),
            reverse=False,
        )
        media_pool.AppendToTimeline(sorted_clips)


append_to_timeline("DAY_013_20230419")

In [ ]:
from src.resolve_init import GetResolve
from pprint import pprint

resolve = GetResolve()
project_manager = resolve.GetProjectManager()
project = project_manager.GetCurrentProject()
media_storage = resolve.GetMediaStorage()
media_pool = project.GetMediaPool()
root_folder = media_pool.GetRootFolder()
current_timeline = project.GetCurrentTimeline()